### STEP 18 : MERGE THE TAG

In [2]:
from connectPyspark import *

:: loading settings :: url = jar:file:/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pc/.ivy2/cache
The jars for the packages stored in: /home/pc/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0d2beb24-89d3-48ed-b7a7-881b554d82c1;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.3.0 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 293ms :: artifacts dl 53ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.3.0 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   

22/10/07 23:19:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 23:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/07 23:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/07 23:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/07 23:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/10/07 23:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/10/07 23:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/10/07 23:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/10/07 23:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
22/10/07 23:19:44 WARN Utils: Service 'SparkUI' could not bind on port 4048. Attempting port 4049.


In [4]:
bmbi = read_avro("output/17_BM_BI_TAG_COST_FINAL")
bmbi

DataFrame[original: string, token: array<string>, tag: array<string>, language: string]

In [5]:
bc = read_avro("output/15_ChineseTag")
bc

DataFrame[original: string, token: array<string>, chinese_tag: array<string>, language: string, tag: array<string>]

In [6]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

merged = bmbi.unionByName(bc, allowMissingColumns=True)

In [7]:
merged.filter(merged.language == "chinese").show()

+-------------------------------------+-----------------------------+--------------------+--------+--------------------+
|                             original|                        token|                 tag|language|         chinese_tag|
+-------------------------------------+-----------------------------+--------------------+--------+--------------------+
|               東西很好 而且 送货很快| [東西, 很, 好, 而且, 送货...|[PROPN, ADV, ADJ,...| chinese| [ns, d, a, c, v, d]|
|             泡麵營養價值哪裡比便當高|   [泡, 麵, 營, 養, 價, 值...|[VERB, ADV, VERB,...| chinese|[v, zg, v, v, n, ...|
| 卖家有送两包面 如果送酸辣粉就更好...|  [卖家, 有, 送, 两包, 面,...|[NOUN, VERB, VERB...| chinese|[n, v, v, m, n, c...|
|            收到的包裹完好无损 棒棒哒|[收到, 的, 包裹, 完好无损,...|[VERB, U, VERB, A...| chinese|[v, uj, v, i, n, zg]|
|  上一個龍哥已經躺了 自己注意健康好嗎| [上, 一個, 龍哥, 已經, 躺...|[VERB, M, NOUN, A...| chinese|[f, m, n, d, v, u...|
|  这个对眼睛真的很好 用了过后 发现...| [这个, 对, 眼睛, 真的, 很...|[PRON, ADP, NOUN,...| chinese|[r, p, n, d, d, a...|
|时刻开始考试考想你想你想你多么多么...| [时刻, 开始, 考试, 考想,

In [19]:
merged.filter(merged.language == "malay").select("token", "tag").show(10, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
|token                                                                                                                                                                                 |tag                                                                                                                                          |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
|[banyak, kali, tan

In [20]:
merged.filter(merged.language == "english").select("token", "tag").show(10, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
|token                                                                                                                                                              |tag                                                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+
|[original, innisfree, masks, cheaper, compared, to, other, sellers]        

In [8]:
shutil.rmtree("output/18_MERGE_TAG")
write_avro(merged, "output/18_MERGE_TAG")

### STEP 19 : PREPARING THE UNIQUE WORDS IN CURRENT SENTENCES

In [9]:
df = read_avro("output/18_MERGE_TAG")

In [10]:
words = \
df.rdd.flatMap(lambda x : x.token) \
.map(lambda x : (x,1)) \
.toDF(["words"]) \
.drop("_2")

words.show()

+-------+
|  words|
+-------+
| banyak|
|   kali|
|  tanya|
| seller|
|  stock|
|   baru|
|   bila|
| sampai|
|    sbb|
|flavour|
|     ni|
|  cepat|
|  habis|
|finally|
|  dapat|
|    pun|
|    beg|
| cantik|
|    ank|
|   suka|
+-------+
only showing top 20 rows



In [11]:
uniqueWords = words.select('words').distinct()

In [12]:
uniqueWords.count()

168433

In [13]:
shutil.rmtree("output/19_uniqueWords")
write_avro(uniqueWords, "output/19_uniqueWords")

### STEP 20 : PREPARING THE UNIQUE WORDS IN CURRENT SENTENCES

In [14]:
uW = read_avro("output/19_uniqueWords")
wL = read_avro("output/7_wcLang.avro")

In [15]:
wL = wL.drop("word_count").drop("id")

In [34]:
wL

DataFrame[word: string, wordLang: string]

In [35]:
uW

DataFrame[words: string]

In [16]:
withPreDetected = uW.join(wL,wL.word ==  uW.words,"left").drop("word")

In [17]:
withPreDetected.show()

+---------+----------------+
|    words|        wordLang|
+---------+----------------+
|    adaaa|  Language.MALAY|
|  alquran|Language.ENGLISH|
|     didi|  Language.MALAY|
|      dri|Language.ENGLISH|
| hantaran|  Language.MALAY|
|   haritu|Language.ENGLISH|
|     hope|Language.ENGLISH|
|    inner|Language.ENGLISH|
|      myb|  Language.MALAY|
|   online|Language.ENGLISH|
|   papepn|  Language.MALAY|
|    patut|  Language.MALAY|
|penghntrn|  Language.MALAY|
|   selagi|  Language.MALAY|
|   sngatt|  Language.MALAY|
|     some|Language.ENGLISH|
|     spai|Language.ENGLISH|
|    still|Language.ENGLISH|
|    tapak|  Language.MALAY|
|    those|Language.ENGLISH|
+---------+----------------+
only showing top 20 rows



In [18]:
nullSet = withPreDetected.filter(withPreDetected.wordLang.isNull())
notNullSet = withPreDetected.filter(~withPreDetected.wordLang.isNull())

In [19]:
nullSet.sample(0.6).show()

+-----------+--------+
|      words|wordLang|
+-----------+--------+
|        Chi|    null|
|        Cui|    null|
|       Dian|    null|
|       Geng|    null|
|       Jiao|    null|
|       Piao|    null|
|         Te|    null|
|       Tiao|    null|
|        Tui|    null|
|        Xin|    null|
|jgkkkkkkkjj|    null|
|     niceps|    null|
|seller3eeee|    null|
|          ~|    null|
|          ่|    null|
|         ァ|    null|
| 一隻小貓咪|    null|
|     一點半|    null|
|       三对|    null|
|   三鮮伊面|    null|
+-----------+--------+
only showing top 20 rows



In [20]:
from linguaLangDetect import *

In [22]:
from pyspark.sql.functions import col, udf
from lingua import Language, LanguageDetectorBuilder
languages = [Language.ENGLISH, Language.CHINESE, Language.MALAY]


nonDistDetector = LanguageDetectorBuilder.from_languages(*languages).build()
withDistDetector = LanguageDetectorBuilder.from_languages(*languages).with_minimum_relative_distance(0.1).build()


def lingua_detect_dist(text):
    global withDistDetector

    try:
        if (withDistDetector == None):
            withDistDetector = LanguageDetectorBuilder.from_languages(*languages).with_minimum_relative_distance(0.1).build()
        return str(withDistDetector.detect_language_of(text))
    
    except:
        return "error"

def lingua_detect(text):
    global nonDistDetector
    
    try:
        if (nonDistDetector == None):
            nonDistDetector = LanguageDetectorBuilder.from_languages(*languages).build()
        return str(nonDistDetector.detect_language_of(text))
    except:
        return "error"    

detectWithDistUDF = udf(lambda z: lingua_detect_dist(z))
detectUDF = udf(lambda z: lingua_detect(z))


corrected = nullSet.withColumn("wordLang", detectWithDistUDF(nullSet.words))

In [23]:
corrected.sample(0.1).show(100, False)

+------------------------------------------------+----------------+
|words                                           |wordLang        |
+------------------------------------------------+----------------+
|~                                               |None            |
|ァ                                              |None            |
|一點半                                          |Language.CHINESE|
|三鮮伊面                                        |Language.CHINESE|
|上六星                                          |Language.CHINESE|
|不差                                            |Language.CHINESE|
|不谢不谢不谢不谢不谢不谢不谢不谢不谢不谢不谢不谢|Language.CHINESE|
|丽雯                                            |Language.CHINESE|
|了兩                                            |Language.CHINESE|
|五層                                            |Language.CHINESE|
|人酸                                            |Language.CHINESE|
|允許                                            |Language.CHINESE|
|冠病                        

In [24]:
union_all(corrected, notNullSet)

DataFrame[words: string, wordLang: string]

In [25]:
shutil.rmtree("output/20_ALL_WORDS_SAMPLE_DETECTED")
write_avro(union_all(corrected, notNullSet), "output/20_ALL_WORDS_SAMPLE_DETECTED")

### STEP 21 : EXAM ABOUTPUNCTUATIONS TOKEN
#### most punctuaton tokens detected is actually mispelled words, however it only is 2541 / 1403676 which is 0.002%, which is neglectible

In [26]:
df = read_avro("output/18_MERGE_TAG")

In [65]:
df.show()

+--------------------+--------------------+--------------------+--------+----------+
|            original|               token|                 tag|language|normal_tag|
+--------------------+--------------------+--------------------+--------+----------+
|banyak kali tanya...|[banyak, kali, ta...|[DET, NOUN, VERB,...|   malay|      null|
|beg cantik ank su...|[beg, cantik, ank...|[NOUN, ADJ, NOUN,...|   malay|      null|
|barang puashati t...|[barang, puashati...|[NOUN, ADJ, NOUN,...|   malay|      null|
|barang sampai dal...|[barang, sampai, ...|[NOUN, VERB, ADP,...|   malay|      null|
|balutan kemas and...|[balutan, kemas, ...|[NOUN, ADJ, CCONJ...|   malay|      null|
|harga berbaloi mu...|[harga, berbaloi,...|[NOUN, ADJ, NOUN,...|   malay|      null|
|kasut dh selamat ...|[kasut, dh, selam...|[NOUN, ADV, ADJ, ...|   malay|      null|
|product ok senang...|[product, ok, sen...|[NOUN, ADJ, ADV, ...|   malay|      null|
|selesa sangat kai...|[selesa, sangat, ...|[ADJ, ADV, NOUN, ...| 

In [75]:
import pyspark.sql.functions as f

punct = "PUNCT"
punctb = "SYM"

def remove_punct_token(token, tag):
    rem= []

    for i in range(len(token)):
        if tag[i] == "PUNCT" or tag[i] == "SYM":
            rem.append(i)
    for i in range(len(rem)):
        rem[i] = rem[i] - i

    for i in rem:
        del token[i]
        del tag[i]


    return token, tag


remUDF = udf(lambda x,y : remove_punct_token(x,y), ArrayType(ArrayType(StringType())))
    

In [27]:
df = read_avro("21_NO_PUNCT_TOKEN")

In [ ]:
df

DataFrame[original: string, token: array<string>, tag: array<string>, language: string, normal_tag: array<string>, corrected: array<array<string>>]

In [ ]:
df\
.withColumn("withpunct", \
            f.array_contains(f.col("tag"),"PUNCT")) \
.filter(f.col("withpunct") == True).select("token", "tag").count()

2541

In [ ]:
df.count()

1403676

### STEP 22 create new language pattern column by replacing the token with languages

In [28]:
replacementDict = read_avro("output/20_ALL_WORDS_SAMPLE_DETECTED")

In [141]:
replacementDict

DataFrame[words: string, wordLang: string]

In [29]:
from pyspark.sql.functions import regexp_replace
corrected  =replacementDict.withColumn("wordLang" ,regexp_replace("wordLang","None|error", "oov"))

In [30]:
corrected.show()

+--------+----------------+
|   words|        wordLang|
+--------+----------------+
|    good|Language.ENGLISH|
|  seller|Language.ENGLISH|
|     the|Language.ENGLISH|
|     and|Language.ENGLISH|
|  barang|  Language.MALAY|
|     you|Language.ENGLISH|
|  sampai|  Language.MALAY|
|delivery|Language.ENGLISH|
|    fast|Language.ENGLISH|
|     for|Language.ENGLISH|
|      tq|Language.ENGLISH|
| terbaik|  Language.MALAY|
|      in|Language.ENGLISH|
|      to|Language.ENGLISH|
|      is|Language.ENGLISH|
|   thank|Language.ENGLISH|
|      ok|  Language.MALAY|
|    baik|  Language.MALAY|
|       i|  Language.MALAY|
|      it|Language.ENGLISH|
+--------+----------------+
only showing top 20 rows



In [31]:
correct_dict = corrected
engSet = correct_dict.filter(correct_dict.wordLang.rlike("Language.ENGLISH"))
bmSet =correct_dict.filter(correct_dict.wordLang.rlike("Language.MALAY"))
bcSet = correct_dict.filter(correct_dict.wordLang.rlike("Language.CHINESE"))
oovSet = correct_dict.filter(~correct_dict.wordLang.rlike("Language.MALAY|Language.CHINESE|Language.ENGLISH"))

In [32]:
oov = set(oovSet.rdd.flatMap(lambda x : x).collect())
bi = set(engSet.rdd.flatMap(lambda x : x).collect())
bm = set(bmSet.rdd.flatMap(lambda x : x).collect())
bc = set(bcSet.rdd.flatMap(lambda x : x).collect())

In [33]:
bi

{'bangles',
 'takot',
 'aduhaiii',
 'moscreep',
 'sikiiit',
 'perrrr',
 'ahhahahhahaha',
 'teleworking',
 'minister',
 'qualityyyyyyy',
 'baii',
 'njpis',
 'serviceiyem',
 'mybaby',
 'tdr',
 'kainpn',
 'pjot',
 'upside',
 'excellentttttt',
 'ssngatt',
 'gampackaging',
 'smuaaaaaaaa',
 'frienddelivery',
 'rennais',
 'soooooooooooo',
 'traveltq',
 'swith',
 'dumortierite',
 'kjgfjvfjn',
 'beutful',
 'approached',
 'direcommended',
 'farmsi',
 'sellertip',
 'permhnn',
 'gile',
 'sellersizing',
 'sukaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa',
 'helpme',
 'buffet',
 'productnice',
 'faceshield',
 'xboring',
 'daythe',
 'james',
 'aspectedrecommended',
 'girdle',
 'lghope',
 'gilaorder',
 'hexpela',
 'tlgrecomanded',
 'á',
 'crash',
 'spender',
 'bly',
 'allsize',
 'wiiii',
 'tetvaik',
 'receivedterbaik',
 'pghntrn',
 'perchance',
 'quaoity',
 'meseller',
 'saldo',
 'ppew',
 'unt',
 'xingtttt',
 'perfecttt',
 'state',
 'solaris',
 'syrup',
 'slmtbeg',
 'jumpsuit',
 'materialc

In [34]:
from pyspark.sql.types import StringType, ArrayType
from pyspark.sql.functions import col, udf, size, split, col

In [35]:
replacement = [bi, bm, bc, oov]
languages = ["en","ms","zh","oov"]

def get_lang(text):
    for i in range(4):
        if text in replacement[i]:
            return languages[i]
    return "oov"

def get_lang_arr(arr):
    return [get_lang(x) for x in arr]


get_lang_udf = udf(lambda x : get_lang_arr(x), ArrayType(StringType()))

In [36]:
df = read_avro("output/18_MERGE_TAG")

In [37]:
df

DataFrame[original: string, token: array<string>, tag: array<string>, language: string, chinese_tag: array<string>]

In [38]:
withLang = df.withColumn("language_pattern", get_lang_udf(col("token")))

In [39]:
shutil.rmtree("output/22__READY_NGRAM")
write_avro(withLang, "output/22__READY_NGRAM")

In [40]:
read_avro("output/22__READY_NGRAM").show()

+--------------------+--------------------+--------------------+--------+-----------+--------------------+
|            original|               token|                 tag|language|chinese_tag|    language_pattern|
+--------------------+--------------------+--------------------+--------+-----------+--------------------+
|taste nice better...|[taste, nice, bet...|[NOUN, ADJ, ADJ, ...| english|       null|[en, en, en, en, ...|
|barang sampai den...|[barang, sampai, ...|[NOUN, ADP, ADP, ...| english|       null|[ms, ms, ms, ms, ...|
|to seller thanks ...|[to, seller, than...|[ADP, NOUN, NOUN,...| english|       null|[en, en, en, en, ...|
|thank you seller ...|[thank, you, sell...|[VERB, PRON, NOUN...| english|       null|[en, en, en, en, ...|
|wow fast delivery...|[wow, fast, deliv...|[PROPN, PROPN, NO...| english|       null|[en, en, en, en, ...|
|good product qual...|[good, product, q...|[PROPN, PROPN, PR...| english|       null|[en, en, en, en, ...|
|unrelated photo a...|[unrelated, pho